# Lesson on Frequencies.

<img src="Jocelyn_Bell.jpg">


Pulsar:
1. ~1.4x as massive as the Sun
1. ~12 km radius (size of Albuquerque)
1. Can produce radio/x-ray/gamma-ray blips every time it spins
1. Spins up to 716 times per second
1. Clock about as good as anything Humans can make


In [ ]:
%matplotlib widget
# The `widget` setting lets you make plots in the output cell, and manipulate zoom, etc.
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import astropy as ap
import astropy.units as u
import sounddevice
from scipy.io import wavfile
import time

In [ ]:
m_ns = 1.4 * u.solMass      # Maybe up to 1.5,1.6, but any heavier and it collapses to a black hole
r_ns = 12 * u.km            # Somewhere in the 10-15 km range

v_ns = (4/3 * np.pi * r_ns ** 3)
d_ns = m_ns / v_ns
print(f"Density of Neutron star: {d_ns.si:.2e}")

In [ ]:
# How much does everybody on Earth weigh, and how does that compare to this:
n_people = 7.7e9 # How many people are there?                                      
m_person = 50 * u.kg # How much does each one weigh, on average? 

m_humanity = n_people * m_person
print(f"Everybody weighs a total of {m_humanity:.1e}")

v_humanity_at_ns_density = m_humanity / d_ns
sidelength = v_humanity_at_ns_density ** (1/3)
print(f"Everybody fits in a {sidelength.to(u.cm):.3f} cube")

## Questions

How large a cube do you need to fit everyone on Earth at normal density?

How many elephants/blue whales/supertankers/heavy thing of your choice would it take to weigh as much as a piece of neutron star material the size of a grain of dust?

In [ ]:
# Your work here

## How do you find pulsars?

Point a radiotelescope at the sky and look for a periodic signal.

## How do you find a periodic signal?

~Plot it out and look for repetition.~

~Turn it into sounds and listen for musical notes, buzzing, or ticking.~

~Magic~

The Fast Fourier Transform.

## What is a Fast Fourier Transform (FFT)?

An extremely clever way of looking for a periodic signal.

It takes a series of data points and tells you how much signal there is at every possible frequency.

It is one of the most widely used algorithms in the world.

## How does it work?

Magic.  (You can understand it, but it is still magic.)

## Then how do you use it?

That's what we are about to learn.

## On pulsars?

Let's start with music.

In [ ]:
# Start with a single tone (C4) played on a flute (see Credits below for references)
fname = 'singleflute-c4.wav'
# Or a simple melody
# fname = 'flute-loop.wav'

In [ ]:
# Load the file
samplerate, data = wavfile.read(fname)
if len(data.shape) == 2:
    # Take only one channel if it is stereo/or multichannel
    data = data[:,0]
print(f"File: {fname} @ {samplerate} samples/second, {len(data)} samples")


In [ ]:
# Play it so we can hear it
sounddevice.play(data, samplerate)
time.sleep(len(data)/samplerate)

In [ ]:
%matplotlib widget
# Let's take a snippet out of the center to reduce the number of samples we worry about
snippetsize = 2 ** 12  # FFT works best when length is power of 2
i_midpoint = (len(data) - snippetsize) // 2    # The // division gives an integer and throws away remainder

snippet = data[i_midpoint:i_midpoint+snippetsize]
print(f"Working with snippet of {len(snippet)} samples = {len(snippet) / samplerate :.2f} seconds")
# Play the snippet so we can hear it
sounddevice.play(snippet, samplerate)
time.sleep(len(snippet)/samplerate)
# Plot the snippet so we can see it
plt.plot(np.arange(len(snippet))/samplerate, snippet)
plt.xlabel("Time (s)")
plt.ylabel("value")

In [ ]:
%matplotlib widget
# Now let's look at the FFT
v_fft = np.fft.rfft(snippet)  # rfft because the input values are real (not complex)
f_fft = np.fft.rfftfreq(len(snippet), 1/samplerate)

plt.plot(f_fft, np.abs(v_fft))  # Why absolute value?  I'll get to that
plt.xlabel("Frequency")
plt.ylabel("Amplitude")

# Why so many peaks?
Because the signal is not a simple sine wave.
A Fourier transform tells you what the sine/cosine wave is at each frequency.

# Let's see what that means by looking at the few most powerful frequencies

In [ ]:
%matplotlib widget
ncomponents = 50
norm = np.sqrt(2)/len(snippet)  # Round trip normalization is unity, but how it gets there varies between libraries
# fig,axlist = plt.subplots(nrows=ncomponents, sharex=True, sharey=True, figsize=(20,20), squeeze=True)
fig,ax = plt.subplots(nrows=1)

# Indices of the highest amplitude components
sorted_i = np.argsort(-np.abs(v_fft))
which_i = sorted_i[:ncomponents] 

# v_fft[which_i[0]] *= complex(1,1)  # What happens if you twist the biggest compent by 45 degrees?

# Times of the snippet samples
tsnip = np.arange(len(snippet))/samplerate
ax.plot(tsnip, snippet, 'k', "original")
ax.set(xlabel="Time (s)", ylabel="value")

reconstruction = np.zeros(len(snippet))
for i in which_i:
    # Calculate the component
    # The value of the FFT component is a complex number where the absolute value
    # is the amplitude at that frequency, and the argument is the phase.
    v = v_fft[i]
    freq = f_fft[i]
    phases = (2 * np.pi * tsnip * freq) + np.angle(v)
    component = np.cos(phases) * np.abs(v) * norm
#     phasor = complex(0,freq * 2 * np.pi)
#     component = norm * (v_fft[i] * np.exp(tsnip * phasor)).real()
    ax.plot(tsnip, component, linewidth=0.5) #, label=f"{freq:.2f} Hz")
    reconstruction += component
    sounddevice.play(reconstruction.astype(snippet.dtype), blocking=True)
    time.sleep(0.25)
    
sounddevice.play(snippet)
    
ax.plot(tsnip, reconstruction, 'b', label=f"{ncomponents} comp. reconst.")
ax.set(xlim=[0.0,0.01])
fig.suptitle(f"With only {ncomponents} sets of (f,a+bi) numbers you can approximate {len(snippet)} audio samples")
ax.legend()

In [ ]:
# Instead of manually reconstructing the signal from individual components, you can use the Inverse FFT to go from frequencies to time.
# Also, I don't seem to have my manual reconstruction quite right
# Start with all zeros of the same shape and type as the FFT output:
v_reconstruct = np.zeros(shape=v_fft.shape, dtype=v_fft.dtype)
# Copy the best ncomponents values
v_reconstruct[which_i] = v_fft[which_i]
reconstruction_fast = np.fft.irfft(v_reconstruct)

ax.plot(tsnip, reconstruction_fast, 'r', label=f"{ncomponents} fast reconst.")
# ax.set(xlim=[0.0,0.01])
# fig.suptitle(f"With only {ncomponents} sets of (f,a+bi) numbers you can approximate {len(snippet)} audio samples")
ax.legend()

In [ ]:
# What frequencies are calculated?
# If you have a 1-second long snippet 10 samples at 0.1 seconds each

np.fft.rfftfreq(n=10, d=0.1)

# The highest frequency is 1 cycle/2 samples (Nyquist Theorem)
# The spacing between frequencies is 1 cycle/full snippet

# How can you use this to pull periodic signals out of noise?

Take a long interval of data, add noise, and see if you can still find the signal

In [ ]:
snippetsize = 2 ** 17 # FFT works best when length is power of 2
i_midpoint = (len(data) - snippetsize) // 2    # The // division gives an integer and throws away remainder

snippet = data[i_midpoint:i_midpoint+snippetsize]
assert len(snippet) == snippetsize
print(f"Working with snippet of {len(snippet)} samples = {len(snippet) / samplerate :.2f} seconds")
# Play the snippet so we can hear it
sounddevice.play(snippet, samplerate)



In [ ]:
# Dilute it with noise by a factor of 10
dilution = 0.1
rms = snippet.std()
noise = np.random.normal(scale=rms, size=snippet.shape)
dilutedsignal = snippet*dilution
noisysignal = noise + dilutedsignal
sounddevice.play(noisysignal.astype(snippet.dtype), samplerate)

tsnip = np.arange(len(snippet))/samplerate

fig,ax = plt.subplots(nrows=1)
ax.plot(tsnip, noisysignal, 'k')
ax.plot(tsnip, dilutedsignal, 'r')
ax.set(xlabel="Time (s)", ylabel="value", xlim=[0.,0.01])



In [ ]:
%matplotlib widget
# Now let's look at the FFT
v_fft = np.fft.rfft(noisysignal)  # rfft because the input values are real (not complex)
f_fft = np.fft.rfftfreq(len(noisysignal), 1/samplerate)

plt.plot(f_fft, np.abs(v_fft)) 
plt.xlabel("Frequency")
plt.ylabel("Amplitude")

# FFT Tips

1. Works on uniformly-spaced data
1. Forward FFT goes from time to frequency, Inverse FFT goes from frequency to time.
1. Algorithm works best for length n a power of 2.  Extremely slow if n has large prime factors.
1. Output frequencies are k/duration for k=-n//2...-2,-1,0,1,2,...n//2
1. Values are complex numbers (real and imaginary correspond to cosine and sine parts)
1. Absolute value is amplitude. amplitude^2 is power.  10*log_10(power) is decibels.
1. If input is real-valued, positive and negative frequencies give same value (so you can discard negative frequencies)
1. Normalization of amplitudes varies from library to library, but for any given library, a round trip doesn't change anything.
1. The zero-frequency value is the average.  For real FFTs this is real, so complex part is sometimes used to store top frequency.


# Credits and references

Jocelyn Bell picture 
https://blog.csiro.au/pioneer-pulsars-pops-into-parkes/

Audio files from \
https://freesound.org

singleflute-c4.wav \
257363__yano1__overall-quality-of-single-note-flute-c4.wav \
https://freesound.org/people/yano1/sounds/257363/

flute-loop.wav \
121089__thirsk__95-flute-loop.wav \
https://freesound.org/people/Thirsk/sounds/121089/

In [ ]:
# Detritus

# Using sympy to solve the simple harmonic oscillator

https://flothesof.github.io/harmonic-oscillator-three-methods-solution.html


%matplotlib widget
# How to plot
plt.plot(np.sin(np.arange(1,100)/10))

import sympy as sy
x,y,z = sy.symbols('x y z')


from sympy import *
x = symbols('x')
a = Integral(cos(x)*exp(x), x)
Eq(a, a.doit())
Eq(Integral(exp(x)*cos(x), x), exp(x)*sin(x)/2 + exp(x)*cos(x)/2)